1. Since 'Part_1.mat' is older version of .mat file (older than version 7.3), it's possible and feasible to use scipy to load file 
2. 'Part_2.mat' is somehow corrupted and there is no way we can open it through python.
3. 'Part_3.mat' was saved as later version (later than version 7.3), so if we want to load the file, we have to use h5py package
4. In 'Part_1.mat' file, there is a cell array whose dimension is 1 x 3000 cells. Each cell contains 3 x N of a patient(N is number of samples) including PPG(1st row), ECG(2nd row), ABP(3rd row).

In [15]:
import scipy.io as sio
Data1 = sio.loadmat( '/data1/Yan-Cheng-Hsu/Preprocessed_Part_1.mat' )# RawData1 is a dictionary whose key is 'Part_1'
Data3 = sio.loadmat( '/data1/Yan-Cheng-Hsu/Preprocessed_Part_3.mat' )

In [16]:
RawData1 = sio.loadmat( '/data1/Yan-Cheng-Hsu/Part_1.mat' )

In [17]:
import sys 
!{sys.executable} -m pip install numpy
#原文網址：https://kknews.cc/code/albn6qx.html
import heartpy as hp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics 

In [18]:
#Data Rearrange and Merge Data1 and Data3
RearrangedData = { 'ppg':[], 'ecg':[], 'abp':[] }#Reaaranged Data1 and Data3
TempList = ['ppg', 'ecg', 'abp']
for i in range( len(Data1['preprocessed_Part_1'][0]) ):
    for j in range( len(Data1['preprocessed_Part_1'][0][0]) ):
        RearrangedData[TempList[j]].append( Data1['preprocessed_Part_1'][0][i][j] )
for i in range( len(Data3['preprocessed_Part_3'][0]) ):
    for j in range( len(Data3['preprocessed_Part_3'][0][0]) ):
        RearrangedData[TempList[j]].append( Data3['preprocessed_Part_3'][0][i][j] )

In [19]:
for i in TempList:
    print( len(RearrangedData[i]) )

6000
6000
6000


In [20]:
#get mean, max, min of RawData1
def getMeanofRawData1():
    Mean = []
    for i in range( len(RawData1['Part_1'][0]) ):
        Mean.append( np.mean( RawData1['Part_1'][0][i][0] ) )
    return Mean
def get_MinandMaxofRawData1():
    Min = []
    Max = []
    for i in range( len(RawData1['Part_1'][0]) ):
        Min.append( min( RawData1['Part_1'][0][i][0] ) )
        Max.append( max( RawData1['Part_1'][0][i][0] ) )
    return min( Min ), max( Max )
def get_StandardDeviationofRawData1():
    Sigma = []
    for i in range( len(RawData1['Part_1'][0]) ):
        Sigma.append( np.std(RawData1['Part_1'][0][i][0]) )
    return Sigma

#Declaration of Attribute Table
AttributeTable = {}
AttributeTable['Sampling_Rate'] = 125.0 #sampling rate = 125Hz
AttributeTable['Meanofppg'] = getMeanofRawData1()
AttributeTable['Sigmaofppg'] = get_StandardDeviationofRawData1()

In [ ]:
AttributeTable['R-peakIndex'] = []
AttributeTable['R-peakValue'] = []
def get_peakIndex_and_peakValue_ofecg( startIndex , endIndex ):
    for i in range( len(RearrangedData['ecg'][startIndex:endIndex]) ):
        #get peak index of ecg
        temp = []
        working_data, measures = hp.process(RearrangedData['ecg'][i], 125.0)
        temp = np.append( working_data['peaklist'], working_data['removed_beats'] )
        temp = np.sort( temp )
        AttributeTable['R-peakIndex'].append( temp )
        
        #get peak value of ecg
        temp2 = []
        temp2 = np.array( temp2 )
        for j in range( len(temp) ):
            temp2 = np.append( temp2 , RearrangedData['ecg'][i][temp[j]] ) 
        AttributeTable['R-peakValue'].append(temp2)
    return
get_peakIndex_and_peakValue_ofecg(0,1000)


#working_data, measures = hp.process(RearrangedData['ecg'][0], 125.0)
#hp.plotter(working_data, measures)
#working_data

In [22]:
for i in range( len(AttributeTable['R-peakIndex']) ):
    print(i,' = ' , len(AttributeTable['R-peakIndex'][i]) )
    
print('=================================================================')
for i in range( len(AttributeTable['R-peakValue']) ):
    print(i,' = ' , len(AttributeTable['R-peakValue'][i]) )

0  =  1041
1  =  1098
2  =  1007
3  =  1170
4  =  1166
5  =  330
6  =  1253
7  =  72
8  =  35
9  =  31
10  =  17
11  =  39
12  =  16
13  =  24
14  =  471
15  =  1079
16  =  990
17  =  395
18  =  1110
19  =  67
20  =  67
21  =  11
22  =  57
23  =  64
24  =  952
25  =  50
26  =  58
27  =  68
28  =  93
29  =  86
30  =  66
31  =  37
32  =  72
33  =  91
34  =  80
35  =  1003
36  =  1014
37  =  650
38  =  210
39  =  816
40  =  971
41  =  958
42  =  151
43  =  25
44  =  82
45  =  94
46  =  912
47  =  40
48  =  53
49  =  10
50  =  150
51  =  11
52  =  56
53  =  26
54  =  13
55  =  38
56  =  12
57  =  51
58  =  24
59  =  11
60  =  12
61  =  36
62  =  146
63  =  107
64  =  927
65  =  14
66  =  1053
67  =  1391
68  =  96
69  =  38
70  =  170
71  =  364
72  =  56
73  =  1852
74  =  130
75  =  28
76  =  191
77  =  61
78  =  480
79  =  2362
80  =  1259
81  =  882
82  =  882
83  =  132
84  =  32
85  =  433
86  =  334
87  =  836
88  =  613
89  =  521
90  =  200
91  =  166
92  =  32
93  =  625
94  =  6

In [ ]:
print( len(RearrangedData['ecg'][96]) )
print('==================================================================')
working_data, measures = hp.process(RearrangedData['ecg'][96], 125.0)
hp.plotter(working_data, measures)
working_data

In [ ]:
AttributeTable['systolicIndex'] = []
AttributeTable['systolicValue'] = []
def get_peakIndex_and_peakValue_ofppg( startIndex , endIndex ):
    for i in range( len(RearrangedData['ppg'][startIndex:endIndex]) ):
        #get peak index of ppg
        temp = []
        working_data, measures = hp.process(RearrangedData['ppg'][i], 125.0)
        temp = np.append( working_data['peaklist'], working_data['removed_beats'] )
        temp = np.sort( temp )
        AttributeTable['systolicIndex'].append( temp )
        
        #get peak value of ppg
        temp2 = []
        temp2 = np.array( temp2 )
        for j in range( len(temp) ):
            temp2 = np.append( temp2 , RearrangedData['ppg'][i][temp[j]] ) 
        AttributeTable['systolicValue'].append(temp2)
    return
get_peakIndex_and_peakValue_ofppg(0,200)

In [ ]:
for i in range( len(AttributeTable['systolicIndex']) ):
    print(i,' = ' , len(AttributeTable['systolicIndex'][i]) )
    
print('=================================================================')
for i in range( len(AttributeTable['systolicValue']) ):
    print(i,' = ' , len(AttributeTable['systolicValue'][i]) )

In [ ]:
print( len(RearrangedData['ppg'][13]) )
print('==================================================================')
working_data, measures = hp.process(RearrangedData['ecg'][13], 125.0)
hp.plotter(working_data, measures)
working_data